In [1]:
!pip install groq langchain langchain-core langchain-groq chromadb pypdf gradio sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain_community import embeddings
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
import os
import time
import textwrap
import gradio as gr

In [5]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/PDF")
text = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(text)

In [9]:
from transformers import AutoModel
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}

embeddings_hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [10]:
vectorstore = Chroma.from_documents(
    documents = chunks,
    collection_name= "groq_embeds",
    embedding = embeddings_hf,
)

retriever = vectorstore.as_retriever()

In [11]:
from google.colab import userdata
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')
llm = ChatGroq(temperature=0, model_name = "mixtral-8x7b-32768" )

In [12]:
from langchain.chains import RetrievalQA

rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": rag_prompt},
)

In [17]:
response = qa_chain.invoke("Who is the captian of Chennai superking?")

In [18]:
print(response)

{'query': 'Who is the captian of Chennai superking?', 'result': 'The captain of Chennai Super Kings is MS Dhoni. This is indirectly indicated in the context in the list of IPL winners where it mentions "Chennai Super King s 4 time s 2010, 2011, 2018, 2021" which implies that MS Dhoni, the captain of Chennai Super Kings, has led the team to four IPL titles in the years 2010, 2011, 2018, and 2021.'}


In [19]:
response['result']

'The captain of Chennai Super Kings is MS Dhoni. This is indirectly indicated in the context in the list of IPL winners where it mentions "Chennai Super King s 4 time s 2010, 2011, 2018, 2021" which implies that MS Dhoni, the captain of Chennai Super Kings, has led the team to four IPL titles in the years 2010, 2011, 2018, and 2021.'

In [20]:
def process_question(user_question):
  response = qa_chain.invoke(user_question)
  full_response = response['result']
  return full_response

In [21]:
interface = gr.Interface(fn= process_question,
                         inputs= gr.Textbox(lines=2, placeholder="Type your question here"),
                         outputs= gr.Textbox(),
                         title= "Groq Chatbot",
                         description="Ask any questio about your documents")

In [22]:
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://dcd767f24c2789def6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
